In [11]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [64]:
## load baseline simulated data
simData = pd.read_excel('../DSSG2023-Heating-Loads-Data/heatPumpCal/baseSim/results_baseCase.xlsx')

In [65]:
simData.head()

,Unnamed: 0,City,Exist_Fuel_Type,Scenario,Census_Area,ANCSA_Region,Util_Name,PCE,Sq_Ft,Capital_Cost,...,IRR,NPV,CO2_lbs_saved,CO2_driving_miles_saved,Fuel_Use_Chg,Fuel_Price_Incremental,Elec_Use_Chg,Elec_Rate_Incremental,Econ,Exist_Fuel
0,0,Adak,1,Base,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.0,...,1.476765,97267.257504,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,Economic,Electricity
1,1,Adak,1,Small Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,678.0,6400.0,...,0.842008,52110.907822,6377.022591,7150.178547,0.0,NaN,-4251.348394,1.237769,Economic,Electricity
2,2,Adak,1,Large Load,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1695.0,6400.0,...,2.221626,150272.882298,15940.709539,17873.375506,0.0,NaN,-10627.139693,1.325889,Economic,Electricity
3,3,Adak,1,Low Elec,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.0,...,1.476765,97267.257504,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.315811,Economic,Electricity
4,4,Adak,1,No PCE,Aleutians West Census Area,Aleut,Adak -TDX Residential,0.7597,1130.0,6400.0,...,1.487922,98061.202974,10628.423185,11917.022775,0.0,NaN,-7085.615457,1.325889,Economic,Electricity


In [72]:
## loading shapefiles for houseunit data
boroughs = gpd.read_file('../DSSG2023-Heating-Loads-Data/shapefiles/akboroughs_2010/')
boroughs = boroughs[['NAME', 'HOUSEUNITS']]
boroughs.rename(columns={'NAME':'Census_Area'}, inplace=True)
boroughs.replace('Petersburg Census Area', 'Petersburg Borough', inplace=True)
boroughs.replace('Anchorage Municipality', 'Anchorage municipality', inplace=True)

## Calculating average estimates

In [145]:
npv_byBoroughs = pd.DataFrame(simData.groupby('Census_Area')['NPV'].mean())
co2_lb_byBoroughs = pd.DataFrame(simData.groupby('Census_Area')['CO2_lbs_saved'].mean())
co2_miles_byBoroughs = pd.DataFrame(simData.groupby('Census_Area')['CO2_driving_miles_saved'].mean())

In [146]:
estimates_byBoroughs = pd.concat([npv_byBoroughs, co2_lb_byBoroughs, co2_miles_byBoroughs],axis=1)

In [147]:
estimates_byBoroughs = estimates_byBoroughs.merge(boroughs, on='Census_Area', how='left')

In [148]:
estimates_byBoroughs['total_NPV'] = estimates_byBoroughs['NPV'] * estimates_byBoroughs['HOUSEUNITS']
estimates_byBoroughs['total_CO2_lbs'] = estimates_byBoroughs['CO2_lbs_saved'] * estimates_byBoroughs['HOUSEUNITS']
estimates_byBoroughs['total_CO2_miles'] = estimates_byBoroughs['CO2_driving_miles_saved'] * estimates_byBoroughs['HOUSEUNITS']

In [149]:
estimates_byBoroughs.head()

,Census_Area,NPV,CO2_lbs_saved,CO2_driving_miles_saved,HOUSEUNITS,total_NPV,total_CO2_lbs,total_CO2_miles
0,Aleutians East Borough,33596.780057,5859.519092,6569.932458,747.0,2.509679e+07,4.377061e+06,4.907740e+06
1,Aleutians West Census Area,25616.350828,6765.392817,7585.635129,1929.0,4.941394e+07,1.305044e+07,1.463269e+07
2,Anchorage municipality,17060.478727,4789.646045,5370.347041,113032.0,1.928380e+09,5.413833e+08,6.070211e+08
3,Bethel Census Area,25797.328485,4337.633246,4863.531803,5919.0,1.526944e+08,2.567445e+07,2.878724e+07
4,Bristol Bay Borough,34642.855508,6107.269706,6847.720579,969.0,3.356893e+07,5.917944e+06,6.635441e+06


In [150]:
estimates_byBoroughs.to_csv('../Output/estimates_byBoroughs.csv', index=False)

## Reading adoption rate estimation

In [151]:
adopt_rates = pd.read_csv('../Output/adoption_scenarios_by_cop_and_pop_max_50p_sat_pretty.csv')

In [152]:
adopt_rates = adopt_rates[['NAME', 'hp_current_prop_b', 'hp_2p_prop_b', 'hp_15p_prop_b']]
adopt_rates.rename(columns={'NAME':'Census_Area'}, inplace=True)

In [154]:
adopt_rates.head()

,Census_Area,hp_current_prop_b,hp_2p_prop_b,hp_15p_prop_b
0,Aleutians East Borough,0.000000,0.064801,0.500000
1,Aleutians West Census Area,0.000000,0.108941,0.500000
2,Anchorage Municipality,0.001129,0.002192,0.035511
3,Bethel Census Area,0.000000,0.006350,0.209225
4,Bristol Bay Borough,0.000000,0.023337,0.500000


In [155]:
estimates_byBoroughs.replace('Anchorage municipality', 'Anchorage Municipality', inplace=True)
estimates_byBoroughs.replace('Wade Hampton Census Area', 'Kusilvak Census Area', inplace=True)

In [158]:
estimates_byBoroughs_wAdopt = estimates_byBoroughs.merge(adopt_rates, on='Census_Area', how='left')

In [161]:
estimates_byBoroughs_wAdopt

,Census_Area,NPV,CO2_lbs_saved,CO2_driving_miles_saved,HOUSEUNITS,total_NPV,total_CO2_lbs,total_CO2_miles,hp_current_prop_b,hp_2p_prop_b,hp_15p_prop_b,total_NPV_current
0,Aleutians East Borough,33596.780057,5859.519092,6569.932458,747.0,2.509679e+07,4.377061e+06,4.907740e+06,0.000000,0.064801,0.500000,0.000000e+00
1,Aleutians West Census Area,25616.350828,6765.392817,7585.635129,1929.0,4.941394e+07,1.305044e+07,1.463269e+07,0.000000,0.108941,0.500000,0.000000e+00
2,Anchorage Municipality,17060.478727,4789.646045,5370.347041,113032.0,1.928380e+09,5.413833e+08,6.070211e+08,0.001129,0.002192,0.035511,2.178042e+06
3,Bethel Census Area,25797.328485,4337.633246,4863.531803,5919.0,1.526944e+08,2.567445e+07,2.878724e+07,0.000000,0.006350,0.209225,0.000000e+00
4,Bristol Bay Borough,34642.855508,6107.269706,6847.720579,969.0,3.356893e+07,5.917944e+06,6.635441e+06,0.000000,0.023337,0.500000,0.000000e+00
5,Denali Borough,15194.269368,4804.294013,5386.770942,1771.0,2.690905e+07,8.508405e+06,9.539971e+06,0.001245,0.017435,0.500000,3.351065e+04
6,Dillingham Census Area,24979.856756,5275.823668,5915.469276,2427.0,6.062611e+07,1.280442e+07,1.435684e+07,0.000000,0.032432,0.500000,0.000000e+00
7,Fairbanks North Star Borough,9512.412164,3723.190045,4174.592956,41783.0,3.974571e+08,1.555660e+08,1.744270e+08,0.002907,0.003469,0.021285,1.155317e+06
8,Haines Borough,14429.327230,5528.442518,6198.715862,1631.0,2.353423e+07,9.016890e+06,1.011011e+07,0.000000,0.072993,0.500000,0.000000e+00
9,Hoonah-Angoon Census Area,18084.270953,5083.958028,5700.341674,1771.0,3.202724e+07,9.003690e+06,1.009531e+07,0.000000,0.097918,0.500000,0.000000e+00


In [163]:
estimates_byBoroughs_wAdopt['total_NPV_current'] = estimates_byBoroughs_wAdopt['total_NPV'] * estimates_byBoroughs_wAdopt['hp_current_prop_b'] 
estimates_byBoroughs_wAdopt['total_NPV_2p'] = estimates_byBoroughs_wAdopt['total_NPV'] * estimates_byBoroughs_wAdopt['hp_2p_prop_b']
estimates_byBoroughs_wAdopt['total_NPV_15p'] = estimates_byBoroughs_wAdopt['total_NPV'] * estimates_byBoroughs_wAdopt['hp_15p_prop_b']

estimates_byBoroughs_wAdopt['total_CO2_lbs_current'] = estimates_byBoroughs_wAdopt['total_CO2_lbs'] * estimates_byBoroughs_wAdopt['hp_current_prop_b']
estimates_byBoroughs_wAdopt['total_CO2_lbs_2p'] = estimates_byBoroughs_wAdopt['total_CO2_lbs'] * estimates_byBoroughs_wAdopt['hp_2p_prop_b']
estimates_byBoroughs_wAdopt['total_CO2_lbs_15p'] = estimates_byBoroughs_wAdopt['total_CO2_lbs'] * estimates_byBoroughs_wAdopt['hp_15p_prop_b']

estimates_byBoroughs_wAdopt['total_CO2_miles_current'] = estimates_byBoroughs_wAdopt['total_CO2_miles'] * estimates_byBoroughs_wAdopt['hp_current_prop_b']
estimates_byBoroughs_wAdopt['total_CO2_miles_2p'] = estimates_byBoroughs_wAdopt['total_CO2_miles'] * estimates_byBoroughs_wAdopt['hp_2p_prop_b']
estimates_byBoroughs_wAdopt['total_CO2_miles_15p'] = estimates_byBoroughs_wAdopt['total_CO2_miles'] * estimates_byBoroughs_wAdopt['hp_15p_prop_b']

In [176]:
state_estimates = estimates_byBoroughs_wAdopt[['total_NPV_current', 'total_NPV_2p', 'total_NPV_15p', 'total_CO2_lbs_current', 'total_CO2_lbs_2p', 'total_CO2_lbs_15p', 'total_CO2_miles_current', 'total_CO2_miles_2p', 'total_CO2_miles_15p']].sum()

In [187]:
state_estimates.rename('estimate', inplace=True)

total_NPV_current          4.087022e+07
total_NPV_2p               1.137467e+08
total_NPV_15p              8.554829e+08
total_CO2_lbs_current      1.264550e+07
total_CO2_lbs_2p           3.579416e+07
total_CO2_lbs_15p          2.585030e+08
total_CO2_miles_current    1.417865e+07
total_CO2_miles_2p         4.013388e+07
total_CO2_miles_15p        2.898442e+08
Name: estimate, dtype: float64

In [189]:
state_estimates.to_csv('../Output/state_estimates.csv')